In [1]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd '/content/drive/My Drive/Colab Notebooks/U-demy - AI Program/AI in Healthcare/Brain_MRI'

/content/drive/My Drive/Colab Notebooks/U-demy - AI Program/AI in Healthcare/Brain_MRI


In [34]:
## Importing required modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile
import cv2
from skimage import io
import tensorflow as tf
from tensorflow.python.keras import Sequential
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from IPython.display import display
from tensorflow.keras import backend as K
from sklearn.preprocessing import StandardScaler, normalize
import os
import random

In [4]:
df = pd.read_csv('data_mask.csv')

In [5]:
mask_df = df[df['mask'] == 1]
mask_df.shape

(1373, 4)

In [6]:
## splitting data up into training and testing
from sklearn.model_selection import train_test_split

X_train, X_val = train_test_split(mask_df, test_size = 0.15)
X_test, X_val = train_test_split(X_val, test_size = 0.5)

In [7]:
print('Training ds: {}'.format(X_train.shape))
print('Validation ds: {}'.format(X_val.shape))
print('Testing ds: {}'.format(X_test.shape))

Training ds: (1167, 4)
Validation ds: (103, 4)
Testing ds: (103, 4)


In [8]:
## The input will be the MRI image and the output will be the mask segmentation 
train_ids = list(X_train.image_path)
train_mask = list(X_train.mask_path)

val_ids = list(X_val.image_path)
val_mask = list(X_val.mask_path)

In [9]:
from utilities import DataGenerator

training_gen = DataGenerator(train_ids, train_mask)
validation_gen = DataGenerator(val_ids, val_mask)

In [37]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPool2D, Activation, Add, UpSampling2D, Concatenate
## Building the residual block
def residual_block(X, filter):

  ## Copying the input
  X_copy = X

  ## Main block
  X = Conv2D(filter, kernel_size = (1, 1), strides = (1, 1), kernel_initializer = 'he_normal')(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)

  X = Conv2D(filter, kernel_size = (3, 3), strides = (1, 1), padding = 'same', kernel_initializer = 'he_normal')(X)
  X = BatchNormalization()(X)

  ## SHhortpath (Due to the dimensions changing with the 2 Conv2D operations)
  X_copy = Conv2D(filter, kernel_size = (1, 1), strides = (1, 1), kernel_initializer = 'he_normal')(X_copy)
  X_copy = BatchNormalization()(X_copy)

  ## Adding both paths together
  X = Add()([X, X_copy])
  X = Activation('relu')(X)

  return X

In [38]:
## Defining the upsampling 
def upsampling_concat(X, encoder_output):
  X = UpSampling2D((2, 2)) (X)
  merge = Concatenate()([X, encoder_output])
  
  return merge

In [42]:

## Building ResUNet
input_size = (256, 256, 3)

X_input = Input(input_size)

## Implementing encoder steps
conv_input = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(X_input)
conv_input = BatchNormalization()(conv_input)
conv_input = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv_input)
conv_input = BatchNormalization()(conv_input)
conv_input_pooling = MaxPool2D(pool_size= (2, 2))(conv_input)

conv2_input = residual_block(conv_input_pooling, 32)
conv2_pooling = MaxPool2D(pool_size= (2, 2))(conv2_input)

conv3_input = residual_block(conv2_pooling, 64)
conv3_pooling = MaxPool2D(pool_size= (2, 2))(conv3_input)

conv4_input = residual_block(conv3_pooling, 128)
conv4_pooling = MaxPool2D(pool_size= (2, 2))(conv4_input)

## Bottleneck 
conv5_input = residual_block(conv4_pooling, 256)

## Transitioning into decoding
decode_1 = upsampling_concat(conv5_input, conv4_input)
decode_1 = residual_block(decode_1, 128)

decode_2 = upsampling_concat(decode_1, conv3_input)
decode_2 = residual_block(decode_2, 64)

decode_3 = upsampling_concat(decode_2, conv2_input)
decode_3 = residual_block(decode_3, 32)

decode_4 = upsampling_concat(decode_3, conv_input)
decode_4 = residual_block(decode_4, 16)

## Adding output convolution
conv_output = Conv2D(1, (1,1), padding = 'same', activation = 'sigmoid')(decode_4)

In [44]:
## Model architecture
model = Model(inputs = X_input, outputs = conv_output)
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_128 (Conv2D)             (None, 256, 256, 16) 448         input_16[0][0]                   
__________________________________________________________________________________________________
batch_normalization_126 (BatchN (None, 256, 256, 16) 64          conv2d_128[0][0]                 
__________________________________________________________________________________________________
conv2d_129 (Conv2D)             (None, 256, 256, 16) 2320        batch_normalization_126[0][0]    
_______________________________________________________________________________________